In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import random
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
random.seed(42)

# Load and prepare the dataset
df = pd.read_csv('epi_r.csv')

# Enhanced data preparation with expanded dietary keywords
def prepare_data(df):
    """Prepare and clean the dataset with more features"""
    
    # Select relevant columns
    required_columns = ['title', 'rating', 'calories', 'protein', 'fat', 'sodium']
    
    # Check for missing required columns and handle gracefully
    available_columns = [col for col in required_columns if col in df.columns]
    if len(available_columns) < len(required_columns):
        print(f"Warning: Some required columns are missing. Using: {available_columns}")
    
    # Create clean dataframe
    clean_df = df[available_columns].copy()
    
    # Handle missing values
    clean_df = clean_df.dropna()
    
    
    # Add meal-time features (extracted from title)
    breakfast_keywords = ['breakfast', 'pancake', 'waffle', 'cereal', 'oatmeal', 'muffin', 
                         'scrambled', 'omelet', 'frittata', 'granola', 'smoothie', 'bagel',
                         'croissant', 'danish', 'morning', 'brunch']
    
    lunch_keywords = ['sandwich', 'salad', 'soup', 'wrap', 'panini', 'pita', 'taco',
                     'burrito', 'quesadilla', 'lunch', 'midday', 'noodle bowl']
    
    dinner_keywords = ['dinner', 'roast', 'steak', 'pasta', 'casserole', 'curry',
                      'stir-fry', 'grilled', 'baked', 'braised', 'stew', 'chili',
                      'lasagna', 'risotto', 'paella', 'supper', 'evening']
    
    # Calculate meal type scores
    clean_df['breakfast_score'] = clean_df['title'].str.lower().apply(
        lambda x: sum(1 for word in breakfast_keywords if word in x))
    clean_df['lunch_score'] = clean_df['title'].str.lower().apply(
        lambda x: sum(1 for word in lunch_keywords if word in x))
    clean_df['dinner_score'] = clean_df['title'].str.lower().apply(
        lambda x: sum(1 for word in dinner_keywords if word in x))
    
    # Improved meal type assignment
    def assign_meal_type(row):
        scores = {
            'breakfast': row['breakfast_score'] * 2 + (2 if row['calories'] < 400 else 0),
            'lunch': row['lunch_score'] * 2 + (2 if 400 <= row['calories'] <= 700 else 0),
            'dinner': row['dinner_score'] * 2 + (2 if row['calories'] > 700 else 0)
        }
        
        # If no clear winner based on keywords, use calories
        if max(scores.values()) == 0:
            if row['calories'] < 400:
                return 'breakfast'
            elif row['calories'] < 700:
                return 'lunch'
            else:
                return 'dinner'
        
        return max(scores, key=scores.get)
    
    clean_df['meal_type'] = clean_df.apply(assign_meal_type, axis=1)
    
    # Enhanced dietary classification
    # Expanded vegetarian keywords
    meat_keywords = ['chicken', 'beef', 'pork', 'fish', 'lamb', 'turkey', 'bacon', 'meat',
                    'sausage', 'ham', 'steak', 'salmon', 'tuna', 'shrimp', 'lobster',
                    'crab', 'duck', 'veal', 'venison', 'prosciutto', 'chorizo', 'seafood',
                    'anchovy', 'sardine', 'halibut', 'cod', 'tilapia', 'meatball', 'ribs']
    
    # Check for vegetarian column or create based on keywords
    if 'vegetarian' in df.columns:
        clean_df['vegetarian'] = df.loc[clean_df.index, 'vegetarian'].fillna(False)
    else:
        clean_df['vegetarian'] = ~clean_df['title'].str.lower().apply(
            lambda x: any(meat in x for meat in meat_keywords))
    
    # Expanded gluten-free keywords
    gluten_keywords = ['pasta', 'bread', 'flour', 'wheat', 'noodle', 'cake', 'cookie',
                      'muffin', 'croissant', 'bagel', 'pizza', 'pie crust', 'pastry',
                      'cracker', 'cereal', 'barley', 'rye', 'couscous', 'bulgur',
                      'semolina', 'spelt', 'farro', 'panko', 'breadcrumb', 'biscuit',
                      'pretzel', 'waffle', 'pancake', 'donut', 'brioche', 'baguette']
    
    # Check for gluten-free column or create based on keywords
    if 'gluten-free' in df.columns:
        clean_df['gluten_free'] = df.loc[clean_df.index, 'gluten-free'].fillna(False)
    elif 'wheat/gluten-free' in df.columns:
        clean_df['gluten_free'] = df.loc[clean_df.index, 'wheat/gluten-free'].fillna(False)
    else:
        clean_df['gluten_free'] = ~clean_df['title'].str.lower().apply(
            lambda x: any(gluten in x for gluten in gluten_keywords))
    
    # Ensure boolean types
    clean_df['vegetarian'] = clean_df['vegetarian'].astype(bool)
    clean_df['gluten_free'] = clean_df['gluten_free'].astype(bool)
    
    return clean_df

# Prepare data
clean_df = prepare_data(df)
print(f"Prepared dataset shape: {clean_df.shape}")
print(f"Vegetarian recipes: {clean_df['vegetarian'].sum()}")
print(f"Gluten-free recipes: {clean_df['gluten_free'].sum()}")

/Users/vincentma/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Prepared dataset shape: (15864, 12)
Vegetarian recipes: 5529
Gluten-free recipes: 3901


In [ ]:
# Optimized neural network for meal type classification
def build_meal_classifier(input_dim):
    """Build an optimized neural network for meal type classification"""
    
    model = keras.Sequential([
        layers.Dense(128, activation='relu', input_shape=(input_dim,)),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(64, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.2),
        layers.Dense(32, activation='relu'),
        layers.Dense(3, activation='softmax')  # 3 classes: breakfast, lunch, dinner
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Train meal type classifier
def train_meal_classifier(clean_df):
    """Train the meal type classification model"""
    
    # Prepare features
    feature_cols = ['calories', 'protein', 'fat', 'sodium', 'rating', 
                   'breakfast_score', 'lunch_score', 'dinner_score']
    
    # Ensure all features exist
    available_features = [col for col in feature_cols if col in clean_df.columns]
    X = clean_df[available_features].values
    
    # Prepare labels
    le = LabelEncoder()
    y = le.fit_transform(clean_df['meal_type'])
    y_categorical = keras.utils.to_categorical(y)
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_categorical, test_size=0.2, random_state=42)
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Build and train model with early stopping
    model = build_meal_classifier(X_train.shape[1])
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    history = model.fit(
        X_train_scaled, y_train,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        callbacks=[early_stopping],
        verbose=0
    )
    
    # Evaluate
    test_loss, test_acc = model.evaluate(X_test_scaled, y_test, verbose=0)
    print(f"Meal classifier accuracy: {test_acc:.3f}")
    
    return model, scaler, le, available_features

# Train the classifier
meal_classifier, meal_scaler, meal_label_encoder, meal_features = train_meal_classifier(clean_df)

In [ ]:
# Optimized neural network for meal compatibility scoring
def build_compatibility_scorer():
    """Build an optimized neural network to score compatibility between meals"""
    
    model = keras.Sequential([
        layers.Dense(256, activation='relu', input_shape=(10,)),  # Adjusted input size
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # Output: compatibility score 0-1
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='mse',
        metrics=['mae']
    )
    
    return model

# Generate training data for compatibility scorer
def generate_compatibility_data(clean_df, n_samples=20000):
    """Generate synthetic training data for meal compatibility"""
    
    X = []
    y = []
    
    for _ in range(n_samples):
        # Randomly select two meals
        idx1, idx2 = np.random.choice(len(clean_df), 2, replace=False)
        meal1 = clean_df.iloc[idx1]
        meal2 = clean_df.iloc[idx2]
        
        # Create feature vector
        features = np.array([
            meal1['calories'], meal1['protein'], meal1['fat'], meal1['rating'],
            meal1['meal_type'] == 'breakfast',
            meal2['calories'], meal2['protein'], meal2['fat'], meal2['rating'],
            meal2['meal_type'] == 'dinner'
        ])
        
        # Create compatibility score (enhanced heuristic)
        compatibility = 0
        
        # Different meal types is good
        if meal1['meal_type'] != meal2['meal_type']:
            compatibility += 0.4
        
        # Complementary nutrition
        total_calories = meal1['calories'] + meal2['calories']
        if 800 <= total_calories <= 1400:
            compatibility += 0.3
        
        # Balanced protein
        total_protein = meal1['protein'] + meal2['protein']
        if 25 <= total_protein <= 60:
            compatibility += 0.2
        
        # Similar ratings
        if abs(meal1['rating'] - meal2['rating']) < 0.5:
            compatibility += 0.1
        
        X.append(features)
        y.append(compatibility)
    
    return np.array(X), np.array(y)

# Train compatibility scorer
X_compat, y_compat = generate_compatibility_data(clean_df)
X_train, X_test, y_train, y_test = train_test_split(
    X_compat, y_compat, test_size=0.2, random_state=42)

compatibility_model = build_compatibility_scorer()

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

compatibility_model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=0
)

test_loss, test_mae = compatibility_model.evaluate(X_test, y_test, verbose=0)
print(f"Compatibility model MAE: {test_mae:.3f}")

Compatibility model MAE: 0.311


In [ ]:
# Enhanced optimization neural network for daily meal selection
class MealPlanOptimizer:
    """Neural network-based meal plan optimizer"""
    
    def __init__(self, clean_df, meal_classifier, compatibility_model):
        self.clean_df = clean_df
        self.meal_classifier = meal_classifier
        self.compatibility_model = compatibility_model
        
    def score_meal_day(self, breakfast, lunch, dinner, 
                      target_calories=2000, target_protein=75, target_fat=65):
        """Score a complete day of meals based on nutritional targets"""
        
        # Calculate nutritional totals
        total_calories = breakfast['calories'] + lunch['calories'] + dinner['calories']
        total_protein = breakfast['protein'] + lunch['protein'] + dinner['protein']
        total_fat = breakfast['fat'] + lunch['fat'] + dinner['fat']
        avg_rating = (breakfast['rating'] + lunch['rating'] + dinner['rating']) / 3
        
        # CRITICAL FIX: Score calories with MUCH HIGHER WEIGHT
        # Use different tolerances for different nutrients
        calorie_score = np.exp(-((total_calories - target_calories) / 150) ** 2)  # Moderate tolerance
        protein_score = np.exp(-((total_protein - target_protein) / 15) ** 2)    # Looser tolerance
        fat_score = np.exp(-((total_fat - target_fat) / 15) ** 2)               # Looser tolerance
        
        # Rating score only matters if nutrition is on target
        nutrition_accuracy = (calorie_score * 0.6 + protein_score * 0.2 + fat_score * 0.2)
        rating_score = (avg_rating / 5.0) * nutrition_accuracy
        
        # Variety score (minimal importance)
        variety_score = 0.95
        
        # HEAVY priority on calories, moderate on protein/fat
        overall_score = (calorie_score * 0.7 +     # 70% weight on calories
                        protein_score * 0.15 +     # 15% weight on protein
                        fat_score * 0.10 +         # 10% weight on fat
                        rating_score * 0.04 +      # 4% weight on ratings
                        variety_score * 0.01)      # 1% weight on variety
        
        return overall_score
    
    def calculate_compatibility(self, meal1, meal2):
        """Calculate compatibility between two meals"""
        
        features = np.array([[
            meal1.calories, meal1.protein, meal1.fat, meal1.rating,
            meal1.meal_type == 'breakfast',
            meal2.calories, meal2.protein, meal2.fat, meal2.rating,
            meal2.meal_type == 'dinner'
        ]])
        
        return self.compatibility_model.predict(features, verbose=0)[0][0]
    
    def generate_meal_plan(self, num_days=3, vegetarian=False, gluten_free=False,
                          target_calories=2000, target_protein=75, target_fat=65):
        """Generate optimized meal plan using neural networks with customizable nutritional targets"""
        
        # Filter recipes based on dietary restrictions
        filtered_df = self.clean_df.copy()
        if vegetarian:
            filtered_df = filtered_df[filtered_df['vegetarian'] == True]
        if gluten_free:
            filtered_df = filtered_df[filtered_df['gluten_free'] == True]
        
        # Check if we have enough recipes
        meal_types = ['breakfast', 'lunch', 'dinner']
        for meal_type in meal_types:
            count = len(filtered_df[filtered_df['meal_type'] == meal_type])
            if count < 5:
                print(f"Warning: Only {count} {meal_type} options available")
        
        meal_plan = []
        used_recipes = set()  # Track used recipes to avoid repetition
        
        for day in range(num_days):
            best_score = -1
            best_meals = None
            
            # Calculate target for each meal type
            breakfast_cal_target = target_calories * 0.25
            lunch_cal_target = target_calories * 0.35
            dinner_cal_target = target_calories * 0.40
            
            # Increase iterations for better accuracy
            for _ in range(300):  # Increased for better chance
                # Select candidates avoiding previously used recipes
                available_df = filtered_df[~filtered_df.index.isin(used_recipes)]
                
                if len(available_df) < 3:
                    available_df = filtered_df  # Reset if not enough unique recipes
                
                try:
                    # CRITICAL FIX: Find meals that match calorie targets FIRST
                    breakfast_options = available_df[
                        (available_df['meal_type'] == 'breakfast') &
                        (available_df['calories'].between(breakfast_cal_target * 0.8, breakfast_cal_target * 1.2))
                    ]
                    
                    lunch_options = available_df[
                        (available_df['meal_type'] == 'lunch') &
                        (available_df['calories'].between(lunch_cal_target * 0.8, lunch_cal_target * 1.2))
                    ]
                    
                    dinner_options = available_df[
                        (available_df['meal_type'] == 'dinner') &
                        (available_df['calories'].between(dinner_cal_target * 0.8, dinner_cal_target * 1.2))
                    ]
                    
                    # If we don't have enough options, progressively widen the range
                    if len(breakfast_options) < 5:
                        breakfast_options = available_df[
                            (available_df['meal_type'] == 'breakfast') &
                            (available_df['calories'].between(breakfast_cal_target * 0.6, breakfast_cal_target * 1.4))
                        ]
                    
                    if len(lunch_options) < 5:
                        lunch_options = available_df[
                            (available_df['meal_type'] == 'lunch') &
                            (available_df['calories'].between(lunch_cal_target * 0.6, lunch_cal_target * 1.4))
                        ]
                    
                    if len(dinner_options) < 5:
                        dinner_options = available_df[
                            (available_df['meal_type'] == 'dinner') &
                            (available_df['calories'].between(dinner_cal_target * 0.6, dinner_cal_target * 1.4))
                        ]
                    
                    # Still not enough? Use all available meals
                    if len(breakfast_options) == 0:
                        breakfast_options = available_df[available_df['meal_type'] == 'breakfast']
                    if len(lunch_options) == 0:
                        lunch_options = available_df[available_df['meal_type'] == 'lunch']
                    if len(dinner_options) == 0:
                        dinner_options = available_df[available_df['meal_type'] == 'dinner']
                    
                    if len(breakfast_options) == 0 or len(lunch_options) == 0 or len(dinner_options) == 0:
                        continue
                    
                    # Sample meals
                    b = breakfast_options.sample(1).iloc[0]
                    l = lunch_options.sample(1).iloc[0]
                    d = dinner_options.sample(1).iloc[0]
                    
                    # Calculate total nutrition
                    total_cal = b.calories + l.calories + d.calories
                    total_prot = b.protein + l.protein + d.protein
                    total_fat = b.fat + l.fat + d.fat
                    
                    # Only consider if calories are within target range
                    if abs(total_cal - target_calories) > target_calories * 0.3:
                        continue  # Skip if calories are off by more than 30%
                    
                    # Calculate compatibility scores
                    bl_compat = self.calculate_compatibility(b, l)
                    ld_compat = self.calculate_compatibility(l, d)
                    bd_compat = self.calculate_compatibility(b, d)
                    
                    # Overall day score with custom targets
                    day_score = self.score_meal_day(
                        {'calories': b.calories, 'protein': b.protein, 'fat': b.fat, 'rating': b.rating},
                        {'calories': l.calories, 'protein': l.protein, 'fat': l.fat, 'rating': l.rating},
                        {'calories': d.calories, 'protein': d.protein, 'fat': d.fat, 'rating': d.rating},
                        target_calories=target_calories,
                        target_protein=target_protein,
                        target_fat=target_fat
                    )
                    
                    # Combined score with even higher weight on nutritional accuracy
                    combo_score = day_score * 0.95 + (bl_compat + ld_compat + bd_compat) / 3 * 0.05
                    
                    if combo_score > best_score:
                        best_score = combo_score
                        best_meals = (b, l, d)
                
                except Exception as e:
                    continue
            
            # Add best combination to meal plan
            if best_meals:
                b, l, d = best_meals
                used_recipes.add(b.name)
                used_recipes.add(l.name)
                used_recipes.add(d.name)
                
                meal_plan.append({
                    'day': day + 1,
                    'meals': {
                        'breakfast': {
                            'title': b.title, 'calories': b.calories, 
                            'protein': b.protein, 'fat': b.fat, 'rating': b.rating
                        },
                        'lunch': {
                            'title': l.title, 'calories': l.calories,
                            'protein': l.protein, 'fat': l.fat, 'rating': l.rating
                        },
                        'dinner': {
                            'title': d.title, 'calories': d.calories,
                            'protein': d.protein, 'fat': d.fat, 'rating': d.rating
                        }
                    },
                    'total_calories': int(b.calories + l.calories + d.calories),
                    'total_protein': round(b.protein + l.protein + d.protein, 1),
                    'total_fat': round(b.fat + l.fat + d.fat, 1),
                    'optimization_score': round(best_score, 3),
                    'target_accuracy': {
                        'calories': round(100 - abs((b.calories + l.calories + d.calories - target_calories) / target_calories * 100), 1),
                        'protein': round(100 - abs((b.protein + l.protein + d.protein - target_protein) / target_protein * 100), 1),
                        'fat': round(100 - abs((b.fat + l.fat + d.fat - target_fat) / target_fat * 100), 1)
                    }
                })
        
        return meal_plan

# Create the optimizer
optimizer = MealPlanOptimizer(clean_df, meal_classifier, compatibility_model)

In [ ]:
# Enhanced display and analysis functions
def display_optimized_meal_plan(meal_plan):
    """Display optimized meal plan with enhanced formatting"""
    
    for day_plan in meal_plan:
        print(f"\n{'='*50}")
        print(f"Day {day_plan['day']} | Score: {day_plan['optimization_score']:.3f}")
        print(f"{'='*50}")
        print(f"Total: {day_plan['total_calories']} cal | {day_plan['total_protein']}g protein | {day_plan['total_fat']}g fat")
        
        # Show accuracy if available
        if 'target_accuracy' in day_plan:
            print(f"Accuracy: Calories {day_plan['target_accuracy']['calories']}% | Protein {day_plan['target_accuracy']['protein']}% | Fat {day_plan['target_accuracy']['fat']}%")
        
        print(f"{'-'*50}")
        
        for meal_type, meal_info in day_plan['meals'].items():
            print(f"\n{meal_type.upper()}: {meal_info['title']}")
            print(f"  {meal_info['calories']} cal | {meal_info['protein']}g protein | {meal_info['fat']}g fat | ⭐ {meal_info['rating']:.1f}")

def analyze_meal_plan_quality(meal_plan):
    """Enhanced analysis of nutritional quality"""
    
    daily_calories = []
    daily_protein = []
    daily_fat = []
    optimization_scores = []
    
    for day in meal_plan:
        daily_calories.append(day['total_calories'])
        daily_protein.append(day['total_protein'])
        daily_fat.append(day['total_fat'])
        optimization_scores.append(day['optimization_score'])
    
    print("\n" + "="*50)
    print("MEAL PLAN ANALYSIS")
    print("="*50)
    print(f"Average Daily Calories: {np.mean(daily_calories):.0f} ± {np.std(daily_calories):.0f}")
    print(f"Average Daily Protein: {np.mean(daily_protein):.1f}g ± {np.std(daily_protein):.1f}g")
    print(f"Average Daily Fat: {np.mean(daily_fat):.1f}g ± {np.std(daily_fat):.1f}g")
    print(f"Average Optimization Score: {np.mean(optimization_scores):.3f}")
    
    # Nutritional guidelines check
    meets_calories = all(1600 <= cal <= 2400 for cal in daily_calories)
    meets_protein = all(45 <= prot <= 100 for prot in daily_protein)
    meets_fat = all(45 <= fat <= 80 for fat in daily_fat)
    
    print(f"\n✓ Meets Calorie Guidelines (1600-2400): {'Yes' if meets_calories else 'No'}")
    print(f"✓ Meets Protein Guidelines (45-100g): {'Yes' if meets_protein else 'No'}")
    print(f"✓ Meets Fat Guidelines (45-80g): {'Yes' if meets_fat else 'No'}")
    print("="*50)

In [ ]:
# Easy-to-use wrapper function
def create_optimized_meal_plan(days=3, vegetarian=False, gluten_free=False,
                             target_calories=2000, target_protein=75, target_fat=65):
    """
    Create an optimized meal plan using neural networks
    
    Parameters:
    - days: Number of days in the plan (default 3)
    - vegetarian: Boolean for vegetarian restriction
    - gluten_free: Boolean for gluten-free restriction
    - target_calories: Daily calorie target (default 2000)
    - target_protein: Daily protein target in grams (default 75)
    - target_fat: Daily fat target in grams (default 65)
    
    Returns:
    - meal_plan: Optimized meal plan
    """
    
    print(f"\nGenerating {days}-day meal plan...")
    print(f"Nutritional targets: {target_calories} calories, {target_protein}g protein, {target_fat}g fat")
    
    if vegetarian:
        print("Dietary restriction: Vegetarian")
    if gluten_free:
        print("Dietary restriction: Gluten-free")
    
    meal_plan = optimizer.generate_meal_plan(
        num_days=days,
        vegetarian=vegetarian,
        gluten_free=gluten_free,
        target_calories=target_calories,
        target_protein=target_protein,
        target_fat=target_fat
    )
    
    display_optimized_meal_plan(meal_plan)
    analyze_meal_plan_quality(meal_plan)
    
    return meal_plan

In [ ]:

# Example usage
print("\n" + "="*50)
print("QUICK START EXAMPLES")
print("="*50)

# Create default 3-day plan (2000 cal, 75g protein, 65g fat)
default_plan = create_optimized_meal_plan()

# Create 3-day vegetarian plan with default nutrition
vegetarian_plan = create_optimized_meal_plan(vegetarian=True)

# Create 3-day gluten-free plan with custom nutrition (1800 cal, 90g protein, 50g fat)
gluten_free_plan = create_optimized_meal_plan(
    gluten_free=True,
    target_calories=1800,
    target_protein=90,
    target_fat=50
)

# Create 5-day low-calorie, high-protein plan
low_cal_plan = create_optimized_meal_plan(
    days=5,
    target_calories=1600,
    target_protein=100,
    target_fat=50
)

# Create 3-day vegetarian and gluten-free plan with custom nutrition
veg_gf_plan = create_optimized_meal_plan(
    vegetarian=True, 
    gluten_free=True,
    target_calories=2200,
    target_protein=70,
    target_fat=80
)


QUICK START EXAMPLES

Generating 3-day meal plan...
Nutritional targets: 2000 calories, 75g protein, 65g fat

Day 1 | Score: 0.860
Total: 1996 cal | 80.0g protein | 105.0g fat
Accuracy: Calories 99.8% | Protein 93.3% | Fat 38.5%
--------------------------------------------------

BREAKFAST: Mozzarella and Tomato Sauce Omelets 
  438.0 cal | 23.0g protein | 34.0g fat | ⭐ 5.0

LUNCH: Roasted Asparagus and Red Onion Quesadillas 
  677.0 cal | 24.0g protein | 40.0g fat | ⭐ 4.4

DINNER: Ditalini Risotto 
  881.0 cal | 33.0g protein | 31.0g fat | ⭐ 5.0

Day 2 | Score: 0.885
Total: 2041 cal | 68.0g protein | 70.0g fat
Accuracy: Calories 98.0% | Protein 90.7% | Fat 92.3%
--------------------------------------------------

BREAKFAST: Cowboy Christmas Breakfast 
  410.0 cal | 24.0g protein | 27.0g fat | ⭐ 3.8

LUNCH: Concord Grape and Champagne Cocktails 
  688.0 cal | 3.0g protein | 1.0g fat | ⭐ 5.0

DINNER: Lamb and Polenta "Lasagne" 
  943.0 cal | 41.0g protein | 42.0g fat | ⭐ 3.8

Day 3 | S